In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 3, Finished, Available)

In [2]:
%%sql
create table if not exists Project_Lakehouse.gold_product
(
Product_ID	long	,
Product_Category	string	,
Product	string	,
Created_TS	timestamp	,
Modified_TS	timestamp	  

)
using delta

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 4, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
Max_Date=spark.sql("""select coalesce(max(Modified_TS),"1900-01-01") from Project_Lakehouse.gold_product""").first()[0]

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 5, Finished, Available)

In [4]:
df_bronze = spark.sql("""select DISTINCT
Product_Category,
Product
from Project_Lakehouse.bronze_sales where Modified_TS>'{}'""".format(Max_Date))

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 6, Finished, Available)

In [5]:
Max_ID = spark.sql("select coalesce(max(Product_ID),0)from Project_Lakehouse.gold_product").first()[0]

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 7, Finished, Available)

In [6]:
df_final=df_bronze.withColumn("Product_ID",Max_ID+monotonically_increasing_id()+1)

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 8, Finished, Available)

In [7]:
df_final.createOrReplaceTempView("ViewProduct")

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 9, Finished, Available)

In [9]:
%%sql
merge into Project_Lakehouse.gold_product as gp
using ViewProduct as vp
on gp.Product=vp.Product and gp.Product_Category = vp.Product_Category
when MATCHED THEN
update SET
gp.Modified_TS = CURRENT_TIMESTAMP()

when not matched THEN
INSERT
(
    gp.Product_ID,
    gp.Product_Category,
    gp.Product,
    gp.Created_TS,
    gp.Modified_TS
)
VALUES
(
    vp.Product_ID,
    vp.Product_Category,
    vp.Product,
    CURRENT_TIMESTAMP(),
    CURRENT_TIMESTAMP()
)

StatementMeta(, e13f6a85-be67-47f8-9520-5e4a0cccbddc, 11, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>